## **Homology Analysis Using BLAST**

This script performs the following operations:

1. Reads gene sequences from FASTA files.  
2. Runs a BLAST search for each gene using the NCBI "nr" database to perform local alignment.  
3. Analyzes BLAST results and filters them based on e-value, identity percentage, and coverage.  
4. Saves the filtered results in a new FASTA file, including details such as e-value, identities, coverage, and species in the sequence description.  

The script uses Biopython for sequence handling and BLAST execution.

In [1]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
import re
import os

def blast_and_filter(gene_names, e_value_threshold=0.01, percent_identity_threshold=30, coverage_threshold=30):
    if not os.path.exists("blast_results"):
        os.makedirs("blast_results")
        
    for name_gene in gene_names:
        # Read sequence and run BLAST
        try:
            if not os.path.exists("genes"):
                os.makedirs("genes")
        
            query_seq = SeqIO.read(f"genes/{name_gene}.fasta", "fasta")
        except FileNotFoundError:
            print(f"File not found for {name_gene}. Skipping...")
            continue

        print(f"Starting BLAST for  {name_gene}...")
        result_handle = NCBIWWW.qblast("blastp", "swissprot", query_seq.seq)
        print(f"BLAST completed for {name_gene}.")

        # Parse and filter results
        blast_records = NCBIXML.parse(result_handle)
        output_path = f"blast_results/{name_gene}_blast.fasta"
        
        with open(output_path, "w") as output_handle:
            for blast_record in blast_records:
                print(f"Number of alignments found for  {name_gene}:", len(blast_record.alignments))
                for alignment in blast_record.alignments:
                    print("Alignment title:", alignment.title)
                    for hsp in alignment.hsps:
                        query_cover = (hsp.align_length / blast_record.query_letters) * 100
                        print(f"HSP: E-value: {hsp.expect}, Identities: {hsp.identities}, "
                              f"Align length: {hsp.align_length}, Query Cover: {query_cover:.2f}%")
                        
                        percent_identity = (hsp.identities / hsp.align_length) * 100
                        if (hsp.expect <= e_value_threshold and
                            percent_identity >= percent_identity_threshold and
                            query_cover >= coverage_threshold):
                            
                            species_match = re.search(r"\[(.*?)\]", alignment.title)
                            species = species_match.group(1) if species_match else "Unknown species"
                            

                            SeqIO.write(
                                SeqIO.SeqRecord(
                                    seq=hsp.sbjct,
                                    id=alignment.accession,
                                    description=f"E-value: {hsp.expect:.2e}, Identities: {hsp.identities}/{hsp.align_length}, "
                                                f"Query Cover: {query_cover:.2f}%, Percent Identity: {percent_identity:.2f}%, "
                                                f"Species: {species}"
                                ),
                                output_handle,
                                "fasta"
                            )
                            break  # Use only the best HSP for each alignment
        
        print(f"Filtered BLAST results for {name_gene} were saved in '{output_path}'")








#### **1: Gene ptsP**

In [2]:
gene_names = ["ptsP"]
blast_and_filter(gene_names)

Starting BLAST for  ptsP...
BLAST completed for ptsP.
Number of alignments found for  ptsP: 50
Alignment title: sp|Q9K8D3.1| RecName: Full=Phosphoenolpyruvate-protein phosphotransferase; AltName: Full=Phosphotransferase system, enzyme I [Halalkalibacterium halodurans C-125]
HSP: E-value: 5.95966e-160, Identities: 228, Align length: 530, Query Cover: 96.89%
Alignment title: sp|O83018.1| RecName: Full=Phosphoenolpyruvate-protein phosphotransferase; AltName: Full=Phosphotransferase system, enzyme I [Bacillus sp. S]
HSP: E-value: 4.75636e-154, Identities: 235, Align length: 530, Query Cover: 96.89%
Alignment title: sp|P42014.1| RecName: Full=Phosphoenolpyruvate-protein phosphotransferase; AltName: Full=Phosphotransferase system, enzyme I [Geobacillus stearothermophilus]
HSP: E-value: 6.92156e-153, Identities: 235, Align length: 530, Query Cover: 96.89%
Alignment title: sp|O69251.1| RecName: Full=Phosphoenolpyruvate-protein phosphotransferase; AltName: Full=Phosphotransferase system, enzyme

c:\Users\catar\anaconda3\envs\bio_env\lib\site-packages\Bio\SeqRecord.py:228: BiopythonDeprecationWarning: Using a string as the sequence is deprecated and will raise a TypeError in future. It has been converted to a Seq object.
  warnings.warn(


#### **2. Gene ButyrylCoA**

In [4]:
gene_names = ["butyrylCoA"]
blast_and_filter(gene_names)

Starting BLAST for  butyrylCoA...
BLAST completed for butyrylCoA.
Number of alignments found for  butyrylCoA: 8
Alignment title: sp|G2SYC0.1| RecName: Full=Butyryl-CoA:acetate CoA-transferase; Short=Butyryl-CoA CoA-transferase [Roseburia hominis A2-183]
HSP: E-value: 0.0, Identities: 332, Align length: 447, Query Cover: 99.78%
Alignment title: sp|B0MC58.1| RecName: Full=Butyryl-CoA:acetate CoA-transferase; AltName: Full=Butyryl-CoA CoA-transferase [Anaerostipes caccae L1-92]
HSP: E-value: 0.0, Identities: 318, Align length: 447, Query Cover: 99.78%
Alignment title: sp|Q0AVM5.1| RecName: Full=Probable butyrate:acetyl-CoA coenzyme A-transferase; Short=Butyrate CoA-transferase [Syntrophomonas wolfei subsp. wolfei str. Goettingen G311]
HSP: E-value: 2.47933e-164, Identities: 229, Align length: 444, Query Cover: 99.11%
Alignment title: sp|P38942.3| RecName: Full=4-hydroxybutyrate coenzyme A transferase [Clostridium kluyveri DSM 555]
HSP: E-value: 7.3562e-97, Identities: 175, Align length: 4

#### **3. Gene MutS**

In [3]:
gene_names = ["MutS"]
blast_and_filter(gene_names)

Starting BLAST for  MutS...
BLAST completed for MutS.
Number of alignments found for  MutS: 7
Alignment title: sp|Q71TF8.1| RecName: Full=Defense against restriction protein B; AltName: Full=DarB [Escherichia phage P1]
HSP: E-value: 2.4347e-18, Identities: 115, Align length: 460, Query Cover: 21.73%
HSP: E-value: 7.66284e-07, Identities: 31, Align length: 94, Query Cover: 4.44%
Alignment title: sp|P52284.1| RecName: Full=Type II methyltransferase M.CvrRI; Short=M.CviRI; AltName: Full=Adenine-specific methyltransferase CviRI; AltName: Full=Modification methylase CviRI [Paramecium bursaria Chlorella virus CV-XZ6E]
HSP: E-value: 9.4596e-08, Identities: 33, Align length: 92, Query Cover: 4.35%
Alignment title: sp|Q53609.1| RecName: Full=Type II methyltransferase M.SalI; Short=M.SalI; AltName: Full=Adenine-specific methyltransferase SalI; AltName: Full=Modification methylase SalI [Streptomyces albus G]
HSP: E-value: 0.00671704, Identities: 34, Align length: 112, Query Cover: 5.29%
Alignment